This file contains the necessary code to get the counts and metrics for each company and then runs it through the various ML models

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
import numpy as np
import csv
import os
import pandas as pd
import math

In [2]:
dir = 'C:/Users/Chaitu Konjeti/socweb-glassdoor-project/REVIEWS/'

In [3]:
all_metrics = {}

metrics_file = pd.read_csv('metrics.csv')

# get all the proper metrics from the metrics file
for row in metrics_file.index:
    #print(int(metrics_file['Data Year - Fiscal'][row]))
    if metrics_file['Company Name'][row] not in all_metrics.keys():
        all_metrics[metrics_file['Company Name'][row]] = []
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])) and not int(metrics_file['Data Year - Fiscal'][row]) != '0':
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))
    else:
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])):
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(all_metrics)

{'AAR CORP': [(304, '2007'), (313, '2008'), (286, '2009'), (367, '2010'), (412, '2011'), (452, '2012'), (453, '2013'), (251, '2014'), (307, '2015'), (344, '2016'), (335, '2017'), (372, '2018')], 'ADC TELECOMMUNICATIONS INC': [(585, '2008'), (396, '2009'), (479, '2010')], 'AFP IMAGING CORP': [(14, '2008')], 'ALPHARMA INC  -CL A': [], 'AMERICAN AIRLINES GROUP INC': [(3534, '2008'), (2982, '2009'), (4032, '2010'), (3602, '2011'), (4326, '2012'), (7628, '2013'), (13139, '2014'), (15574, '2015'), (14485, '2016'), (13945, '2017'), (13176, '2018')], 'CECO ENVIRONMENTAL CORP': [(45, '2008'), (32, '2009'), (33, '2010'), (39, '2011'), (43, '2012'), (62, '2013'), (87, '2014'), (112, '2015'), (138, '2016'), (115, '2017'), (114, '2018')], 'ASA GOLD AND PRECIOUS METALS': [(5, '2008'), (0, '2009'), (0, '2010'), (2, '2011'), (1, '2012'), (0, '2013'), (-1, '2014'), (-1, '2015'), (-2, '2016'), (-1, '2017'), (-1, '2018')], 'AVX CORP': [(346, '2007'), (297, '2008'), (335, '2009'), (505, '2010'), (438, '20

In [5]:
metrics = []
list_of_posts = []

#iterate through all files in the directory
for filename in os.listdir(dir):
    print(filename)
    df = pd.read_csv(dir + filename, header=0)

    #get dates and pros columns from data file
    dates = df['date']
    pros = df['pros']

    #get the list of years for all the posts in the file and create a dictionary with years as the keys
    years = [date.split(' ')[3] for date in dates]
    corpus = {key: '' for key in set(years)}

    #add the post to the proper year in the dictionary
    for date, pro in zip(years, pros):
        corpus[date] += pro + ' '

    #sort the keys in order
    keys = sorted(corpus)
    
    company_name = filename.split('.')[0]
    
    metric_years = [val[1] for val in set(all_metrics[company_name]) if val[1] in years]
    
    metrics.extend([val[0] for val in set(all_metrics[company_name]) if val[1] in years])
    
    #only use the metrics from years that present in the datafile
    for key in keys:
        if key in metric_years:
            list_of_posts.append(corpus[key])
    
#creates count vectorizer for posts
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 3))
counts = vectorizer.fit_transform(list_of_posts)

metrics = np.asarray(metrics)

AMERICAN AIRLINES GROUP INC.csv
APPLE INC.csv
AVNET INC.csv


In [6]:
print(counts)

  (0, 124)	1
  (0, 1127)	1
  (0, 305)	6
  (0, 1740)	1
  (0, 765)	3
  (0, 910)	2
  (0, 4297)	1
  (0, 4598)	1
  (0, 5935)	1
  (0, 3686)	1
  (0, 4892)	1
  (0, 100)	1
  (0, 6804)	3
  (0, 1665)	1
  (0, 2159)	1
  (0, 290)	1
  (0, 2560)	1
  (0, 4110)	1
  (0, 4959)	2
  (0, 5875)	1
  (0, 8106)	1
  (0, 4332)	1
  (0, 1030)	1
  (0, 2780)	1
  (0, 4097)	1
  :	:
  (26, 425)	1
  (26, 3165)	1
  (26, 5392)	1
  (26, 2540)	1
  (26, 7834)	1
  (26, 6381)	1
  (26, 74)	1
  (26, 7401)	1
  (26, 6165)	1
  (26, 4692)	1
  (26, 1418)	1
  (26, 4902)	1
  (26, 1507)	1
  (27, 6599)	1
  (27, 1589)	1
  (27, 3816)	1
  (27, 2134)	1
  (27, 6620)	1
  (27, 1590)	1
  (27, 3817)	1
  (27, 6600)	1
  (27, 2135)	1
  (27, 1591)	1
  (27, 3818)	1
  (27, 6601)	1


In [7]:
print(counts.shape)

(28, 8511)


In [8]:
print(metrics)

[ 13176   4326  14485   7628   2982  13139  13945   4032  15574   3602
   3534  26616  78483  93515  11581 112506  97586  71867  45640  17906
  71061 104126   2666   3340   2524   3100   3362   2762]


In [9]:
print(metrics.shape)

(28,)


In [10]:
clf = RandomForestClassifier(max_depth=5)
clf.fit(counts, metrics)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
pred = []
for i in range(counts.shape[1]):
    if i % 2 == 0:
        pred.append(1)
    else:
        pred.append(1)
print(clf.predict([pred]))

[2524]
